# Openai Assistants
> Functionality for making calls to OpenAI Assistants

In [ ]:
#| default_exp assistant
import time

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os
import getpass

In [ ]:
#| export
class OpenAIAssistantManager:
    def __init__(self, client):
        self.client = client
        self.current_assistant = None
        self.current_thread = None

    def add_file(self, file_path, purpose='assistants'):
        with open(file_path, "rb") as file_data:
            return self.client.files.create(file=file_data, purpose=purpose)

    def create_assistant(self, name, description, model, tools, file_id):
        self.current_assistant = self.client.beta.assistants.create(
            name=name,
            description=description,
            model=model,
            tools=tools,
            file_ids=[file_id]
        )
        return self.current_assistant

    def create_thread(self, user_message, file_id):
        self.current_thread = self.client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    "content": user_message,
                    "file_ids": [file_id]
                }
            ]
        )
        return self.current_thread

    def run_assistant(self, model=None, instructions=None, tools=None):
        from time import sleep

        if self.current_thread is None or self.current_assistant is None:
            raise Exception("Assistant and Thread must be initialized before running.")

        run_parameters = {
            "thread_id": self.current_thread.id,
            "assistant_id": self.current_assistant.id
        }

        # Adding optional parameters if they are provided
        if model is not None:
            run_parameters["model"] = model
        if instructions is not None:
            run_parameters["instructions"] = instructions
        if tools is not None:
            run_parameters["tools"] = tools

        run = self.client.beta.threads.runs.create(**run_parameters)

        run = self.client.beta.threads.runs.retrieve(thread_id=self.current_thread.id, run_id=run.id)

        print(self.client.beta.threads.messages.list(thread_id=self.current_thread.id))
        
        print(run)
        counter = 0
        while run.status != "completed":
            run = self.client.beta.threads.runs.retrieve(thread_id=self.current_thread.id, run_id=run.id)
            if counter % 10 == 0:
                print(f"\t\t{run}")
            counter += 1
            sleep(5)

        return self.client.beta.threads.messages.list(thread_id=self.current_thread.id)


    def send_message(self, message_content):
        if self.current_thread is None:
            raise Exception("No active thread. Create a thread first.")
        return self.client.beta.threads.update(
            thread_id=self.current_thread.id,
            updates=[
                {
                    "role": "user",
                    "content": message_content
                }
            ]
        )


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()